## ضروری لائبریریاں درآمد کریں  
PyTorch اور transformers لائبریریاں درآمد کریں جو Phi-4 ماڈل کو لوڈ اور استعمال کرنے کے لیے ضروری ہیں۔  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## بے ترتیب بیج مقرر کریں  
بے ترتیب بیج مقرر کریں تاکہ مختلف بار چلانے پر ایک جیسے نتائج حاصل کیے جا سکیں۔  


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning ماڈل اور ٹوکنائزر لوڈ کریں
Microsoft Phi-4-mini-flash-reasoning ماڈل اور اس کے متعلقہ ٹوکنائزر کو Hugging Face سے لوڈ کریں۔ ماڈل کو تیز تر تجزیہ کے لیے CUDA پر لوڈ کیا جائے گا۔


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## ان پٹ پیغام تیار کریں  
ایک گفتگو کا پیغام بنائیں جس میں ایک مربع مساوات کا ریاضی مسئلہ ہو اور اسے ماڈل کے چیٹ ٹیمپلیٹ کے مطابق ترتیب دیں۔


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## جواب تیار کریں  
ماڈل سے جواب تیار کریں، دیے گئے پیرامیٹرز جیسے temperature اور top_p کا استعمال کرتے ہوئے تاکہ نتیجے میں بے ترتیبی کو کنٹرول کیا جا سکے۔


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## آؤٹ پٹ کو متن میں تبدیل کریں  
پیدا کردہ ٹوکن سیکوئنسز کو دوبارہ انسان کے پڑھنے کے قابل متن میں تبدیل کریں، اصل ان پٹ ٹوکنز کو چھوڑ کر تاکہ صرف ماڈل کا جواب دکھایا جا سکے۔


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
